In [14]:
# Got dataset used in this notebook from: Learning Multiple Layers of Features from Tiny Images, Alex Krizhevsky, 2009.

In [15]:
import torch
from torch.nn import functional as F

In [ ]:
# Get data
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

# Get training data
x_train = torch.tensor([], dtype=float)
y_train = torch.tensor([], dtype=float)
for i in range(1, 6):
    file = f"../data/CifarTen/data_batch_{i}"
    batch_dict = unpickle(file)
    batch_data = torch.tensor(batch_dict[b"data"].astype(float))
    label_data = torch.tensor(batch_dict[b"labels"])
    x_train = torch.cat((x_train, batch_data))
    y_train = torch.cat((y_train, label_data))

# Get test data
file = f"../data/CifarTen/test_batch"
batch_dict = unpickle(file)
x_test = torch.tensor(batch_dict[b"data"].astype(float))
y_test = torch.tensor(batch_dict[b"labels"])

In [ ]:
x_train = x_train.reshape((-1, 3, 32, 32)) # Image size
x_test = x_test.reshape((-1, 3, 32, 32))

AttributeError: 'torch.Size' object has no attribute 'value'